In [ ]:
!pip install htrc-feature-reader
# !pip install htrc
# !pip install isbnlib

In [ ]:
from lxml import etree
import os
from urllib.request import urlopen
import json
from htrc_features import *
from htrc import workset, metadata
import pandas as pd
from lxml import etree
from time import sleep
from tqdm import tqdm
from isbnlib import mask, to_isbn10, is_isbn10, is_isbn13
import re
import linecache
import time
import json

def get_link(link):
    r = requests.get(link)
    return(r)

def get_htid(link):
    r = get_link(link)
    data = r.json()
    if len(data["items"]) > 0:
      matched = True
      htids = [(value['htid']) for value in data["items"]]
      year = [(value['publishDates']) for value in data["records"].values()][0]
      title = [(value['titles']) for value in data["records"].values()][0]
    else:
      matched = False
      htids = []
      year = 0
      title = ""
    return(htids, year, title, matched)

def get_subject(link):
    r = get_link(link)
    data = r.json()
    result = [(value['marc-xml']) for value in data["records"].values()][0]
    result = result.replace('encoding="UTF-8"', "")
    root = etree.fromstring(result)
    subject_list = []
    for record in root:
      fields = ['600', '610', '611', '630', '650', '651', '655']
      subject_no = 0
      for field in fields:
        query = "".join(['{http://www.loc.gov/MARC21/slim}datafield[@tag="', field, '"]'])
        subjects = record.findall(query)
        for item in subjects:
          subfields = item.findall('{http://www.loc.gov/MARC21/slim}subfield')
          Ind2 = item.get("ind2")
          if Ind2 == "0":
            for s in subfields:
              if s.get('code') == 'a':
                  value = s.text
                  subject_list.append(value)
    return([subject_list, subject_no])


# Open file and read line by line
file1 = open('/content/classics_list.txt', 'r')

count = sum( 1 for line in file1 )
print(count)

In [ ]:
data_summary = []
htid_lists = []
subject_lists = []

for i in tqdm(range(1, count)):
    # Get next line
    url = linecache.getline('/content/classics_list.txt', i)
    json_file_name = url.split("/")[5].split(".")[0]

    # if line is empty
    # end of file is reached
    if not url:
        break

    # store the response of URL
    response = urlopen(url)

    # storing the JSON response
    # from url in data
    data_json = json.loads(response.read())
    if data_json['availability']['Open Library']:
        open_lib_url = data_json['availability']['Open Library']
        print(open_lib_url)

        #https://openlibrary.org/search.json
        new_open_lib_url = open_lib_url.replace('search?', 'search.json?')

        r = requests.get(new_open_lib_url)
        ol_data = r.json()
        ol_filename = "".join(['./open_library_json/', json_file_name, '.json'])
        try:
          with open(ol_filename, 'w+') as file:
            json.dump(ol_data, file)
        except FileExistsError:
          print(f"The file '{file_path}' already exists.")


        # store the response of URL
        response = urlopen(new_open_lib_url)
        data_json = json.loads(response.read())

        try:
          rnumbers = data_json['docs'][0]['id_hathi_trust']
        except:
          rnumbers = []
        try:
          isbns = data_json['docs'][0]['isbn']
        except:
          isbns = []

        rnumber_count = len(rnumbers)
        isbn_count = len(isbns)

        if rnumber_count > 0:
          print(rnumbers[0])
          link = "".join(["https://catalog.hathitrust.org/api/volumes/full/recordnumber/",
                          rnumbers[0], ".json"])
        elif isbn_count > 0:
          print(isbns[0])
          link = "".join(["https://catalog.hathitrust.org/api/volumes/full/isbn/",
                          isbns[0], ".json"])
        print(link)
        htids = get_htid(link)

        r = requests.get(link)
        data = r.json()
        ht_filename = "".join(['./hathitrust_json/', json_file_name, '.json'])
        try:
          with open(ht_filename, 'w+') as file:
            json.dump(data, file)
        except FileExistsError:
          print(f"The file '{file_path}' already exists.")

        data_summary.append({
            "osurl":url,
            "ht_link":link,
            "ht_matched":htids[3],
            "htid_count":len(htids[0]),
            "year":htids[1],
            'title':htids[2],
            "subject_no":subjects[1]
        })

        if htids[3] == True:

          subjects = get_subject(link)

          for item in htids[0]:
              htid_lists.append({
                  "osurl":url,
                  "ht_link":link,
                  "htid":item
              })
          for item in subjects[0]:
            subject_lists.append({
                "osurl":url,
                "subject":item
            })


        time.sleep(2)


print(data_summary)
print(htid_lists)
print(subject_lists)

file1.close()




In [ ]:
data_summary_df = pd.DataFrame(data_summary)

print(len(data_summary))
print(sum(data_summary_df['ht_matched']))
print(len(htid_lists))
print(len(subject_lists))

In [ ]:
data_summary_df.head()

In [ ]:
htid_df = pd.DataFrame(htid_lists)
htid_df.head()

In [ ]:
subject_df = pd.DataFrame(subject_lists)
subject_df.head()

In [120]:
data_summary_df.to_csv("data_summary.csv")
htid_df.to_csv("htid_list.csv")
subject_df.to_csv("subject_list.csv")